# РЕШЕНИЕ ДЛЯ УПРАЖНЕНИЙ

Обновите проект "день 1", чтобы на веб-странице "Итоги" использовалась модель с открытым исходным кодом, работающая локально через Allama, а не OpenAI

Вы сможете использовать этот метод для всех последующих проектов, если предпочитаете не использовать платные API.

**Преимущества:**
1. Без изменений API - интерфейс с открытым исходным кодом
2. Данные не покидают ваш ящик

**Недостатки:**
1. Значительно меньшая мощность, чем у модели Frontier

## Краткое описание установки Ollama

Просто посетите [ollama.com](https://ollama.com) и установите!

После завершения работы сервер llama уже должен быть запущен локально.  
Если вы посетите:  
[http://localhost:11434/](http://localhost:11434/)

Вы должны увидеть сообщение "Обама запущен".  

Если нет, откройте новый терминал (Mac) или Powershell (Windows) и введите `ollama serve`  
Затем попробуйте [http://localhost:11434/](http://localhost:11434/) еще раз.

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [ ]:
# Constants

MODEL = "llama3.2"

In [ ]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# Let's try one out

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

## Типы подсказок

Возможно, вы уже знаете об этом, но если нет, то вам предстоит с этим ознакомиться!

Такие модели, как GPT4o, были обучены получать инструкции определенным образом.

Они ожидают, что будут получать:

**Системное приглашение**, которое сообщает им, какую задачу они выполняют и какой тон следует использовать

** Пользовательское приглашение** - начало разговора, на которое они должны ответить.

In [ ]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from Ollama expects the same message format as OpenAI:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [ ]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Пришло время объединить усилия - теперь с Ollama вместо OpenAI

In [ ]:
# And now: call the Ollama function instead of OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [ ]:
summarize("https://edwarddonner.com")

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://edwarddonner.com")

# Давайте попробуем другие веб-сайты

Обратите внимание, что это будет работать только с веб-сайтами, которые можно очистить, используя этот упрощенный подход.

Веб-сайты, отображаемые с помощью Javascript, такие как приложения React, отображаться не будут. Смотрите папку community-contributions для реализации Selenium, которая позволяет обойти эту проблему. Вам нужно будет прочитать об установке Selenium (спросите у ChatGPT!)

Также веб-сайты, защищенные с помощью CloudFront (и подобных), могут выдавать 403 ошибки - большое спасибо Энди Джею за указание на это.

Но многие веб-сайты будут работать просто отлично!

In [ ]:
display_summary("https://cnn.com")

In [ ]:
display_summary("https://anthropic.com")

# Поделитесь своим кодом

Я был бы рад, если бы вы впоследствии поделились своим кодом, чтобы я мог поделиться им с другими! Вы заметите, что некоторые студенты уже внесли изменения (включая реализацию Selenium), которые вы найдете в папке "Вклады сообщества". Если вы хотите добавить свои изменения в эту папку, отправьте запрос на добавление ваших новых версий в эту папку, и я объединю ваши изменения.

Если вы не являетесь экспертом в git (а я им не являюсь!), то БОГ дал несколько полезных инструкций о том, как отправлять запрос на удаление. Это довольно сложный процесс, но как только вы его выполните, все станет совершенно ясно. Совет профессионала: лучше всего, если вы очистите выходные данные своих записных книжек Jupyter (Отредактируйте >> Очистите выходные данные всех ячеек, а затем сохраните) для чистых записных книжек.

Инструкции по рекламе любезно предоставлены другом с искусственным интеллектом: https://chatgpt.com/share/670145d5-e8a8-8012-8f93-39ee4e248b4c